# SICOM Face analysis
In this lab we will process and play around with images of faces.

The images used in this lab were extracted from the "trombinoscope" coming as a pdf file. They are shown below.
As you can see the images show portraits with some differences in terms of size, head pose, quality of the image etc.
<table><tr><td><img src="data/page-1.png" style="width: 300px;"/></td><td><img src="data/page-2.png" style="width: 300px;"/></td><td><img src="data/page-3.png" style="width: 300px;"/></td></tr></table>

## Goal
The goal of this lab is to perform some automatic operations on the images presented above.
1. Extract automatically the faces from an image
2. Extract face landmarks (e.g., positions of eyes, mouth and nose)
3. Align faces (geometrically transform images of faces)
4. Segment faces

## Set-up
This lab will mainly rely on the following python libraries:
- `open-cv`
- `scikit-image`
- `dlib`

You can install these libraries with `pip install name_of_the_lib`.

In [ ]:
# ! pip install opencv-python
# ! pip install scikit-image
# ! pip install dlib


## Convenience functions

In [ ]:
# This function show an image using the matplotlib or opencv backend. You can change the default flag in the function.

import matplotlib.pyplot as plt
import cv2
import numpy as np
import cv2

# Plot in an external window. The plot will be interactive.
# Options for interactive plots are 'osx', 'qt4', 'qt5', 'gtk3', 'wx', 'qt', 'gtk', 'tk'...
# %matplotlib widget 
#%matplotlib notebook

# Plot inline, within the notebook.
%matplotlib inline

def show_img(img, img_name="fig", flag='matplt'):
    if flag== 'cv':
        cv2.imshow('img',img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    if flag == 'matplt': 
        plt.figure()
        plt.imshow(img, cmap=plt.cm.gray)
        plt.title(img_name)

## Set-up output folder

Results will be stored in a `output` folder in the current directory. Results for each part of the lab will be stored in a dedicated directory.

In [ ]:
import os

outpath = os.path.join(".","output")
# create dir
if not os.path.exists(outpath):
    os.makedirs(outpath)
    
print("Output files will be saved in:", outpath)

# Part 1 - Extract faces
The first task we address is to crop the faces from the image. The goal is to generate a collection of images each one containing a face.

Below, an example of the first three images extracted.

<table><tr><td><img src="examples/face_00.png" style="width: 200px;"/></td><td><img src="examples/face_01.png" style="width: 200px;"/></td><td><img src="examples/face_02.png" style="width: 200px;"/></td></tr></table>

### Question
Define an algorithm for automatically extracting faces from the images. Implement it, run it on the input images in order to save the faces as separate images and explain your choices.

You can use for example a pre-trained face detector. Open-CV offers an implementation of the face detection algorithm proposed by Viola and Jones, which is based on Haar features (a family of wavelets).
See the corresponding tutorial.
https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html
Attention that you may need to reduce the size of the image as it can take long time.

Using a face detector can be a good strategy when processing a natural image containing faces. In our case, the image shows a structure that can be exploited. Here, each face is separated from the others and corresponds to a specific rectangular region in the image. You can exploit this fact for extracting the portraits from the image (withouth detecting explicitly the faces). In this case the alpha channel (4th channel) can be of interest.

Other strategies can also be possible.


In [ ]:
# function for extracting faces from an image
# Take an image as input and give a list of images of faces

import numpy as np
from skimage import morphology
from skimage.measure import label, regionprops

def extract_faces(img):
    img_faces = []
    # write your code here
    # return a list of images (could be of different size), each one containing a portrait
    return img_faces

In [ ]:
import cv2

img_files = ['data/page-96dpi-1.png', 'data/page-96dpi-2.png', 'data/page-96dpi-3.png']

# set output folder
outpath_folder = os.path.join(outpath,"faces")
# create dir
if not os.path.exists(outpath_folder):
    os.makedirs(outpath_folder)

# this variable will contain the list of face images extracted from the input images
img_faces = []

# cycle on the three input images
for file in img_files:
    # read the image
    img = cv2.imread(file, cv2.IMREAD_UNCHANGED)
    
    # extract faces and append them to the list
    img_faces += extract_faces(img)
    
    
print('Extracted %d faces' % len(img_faces))    


# show the extracted faces
#for f in img_faces:
    #print(f.shape)
    #show_img(f,flag="cv")   
    
# show the extracted faces
i = 0
for f in img_faces:
    cv2.imwrite((outpath_folder+'/face_%02d.png' % i), f)
    i = i+1
    


# Part 2 - Face landmarks
The goal of this part is to process each face image extracted in the previous part and extract face landmarks

#### Download the pre-trained model here 

http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

- Place this file in your default ipython notebook folder -> it is already available in `/data`


Below, an example of the first three images with extracted landmarks superimposed.

<table><tr><td><img src="examples/face_00_landmarks.png" style="width: 200px;"/></td><td><img src="examples/face_01_landmarks.png" style="width: 200px;"/></td><td><img src="examples/face_02_landmarks.png" style="width: 200px;"/></td></tr></table>


## Setup
Define some functions and utilities based on `dlib`

In [ ]:
import cv2
import dlib
import numpy
from time import sleep
import sys

PREDICTOR_PATH = "data/shape_predictor_68_face_landmarks.dat"
SCALE_FACTOR = 1 
FEATHER_AMOUNT = 11

FACE_POINTS = list(range(17, 68))
MOUTH_POINTS = list(range(48, 61))
RIGHT_BROW_POINTS = list(range(17, 22))
LEFT_BROW_POINTS = list(range(22, 27))
RIGHT_EYE_POINTS = list(range(36, 42))
LEFT_EYE_POINTS = list(range(42, 48))
NOSE_POINTS = list(range(27, 35))
JAW_POINTS = list(range(0, 17))

# Points used to line up the images.
ALIGN_POINTS = (LEFT_BROW_POINTS + RIGHT_EYE_POINTS + LEFT_EYE_POINTS +
                               RIGHT_BROW_POINTS + NOSE_POINTS + MOUTH_POINTS)

# Points from the second image to overlay on the first. The convex hull of each
# element will be overlaid.
OVERLAY_POINTS = [
    LEFT_EYE_POINTS + RIGHT_EYE_POINTS + LEFT_BROW_POINTS + RIGHT_BROW_POINTS,
    NOSE_POINTS + MOUTH_POINTS,
]

# Amount of blur to use during colour correction, as a fraction of the
# pupillary distance.
COLOUR_CORRECT_BLUR_FRAC = 0.6

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)


In [ ]:
# Functions from code from the book "Master-Computer-Vision-OpenCV3-in-Python-and-Machine-Learning" 
# https://github.com/PacktPublishing/Master-Computer-Vision-OpenCV3-in-Python-and-Machine-Learning
class TooManyFaces(Exception):
    pass

class NoFaces(Exception):
    pass

def get_landmarks(im):
    # Returns facial landmarks as (x,y) coordinates
    rects = detector(im, 1)
    
    if len(rects) > 1:
        raise TooManyFaces
    if len(rects) == 0:
        raise NoFaces

    return numpy.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])

def get_landmarks1(im):
    # Returns facial landmarks as (x,y) coordinates
    rects = detector(im, 1)
    
    return numpy.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])


def annotate_landmarks(im, landmarks):
    #Overlays the landmark points on the image itself
    
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(im, str(idx), pos,
                    fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    fontScale=0.4,
                    color=(0, 0, 255))
        cv2.circle(im, pos, 3, color=(0, 255, 255))
    return im
def draw_convex_hull(im, points, color):
    points = cv2.convexHull(points)
    cv2.fillConvexPoly(im, points, color=color)

def get_face_mask(im, landmarks):
    im = numpy.zeros(im.shape[:2], dtype=numpy.float64)

    for group in OVERLAY_POINTS:
        draw_convex_hull(im,
                         landmarks[group],
                         color=1)

    im = numpy.array([im, im, im]).transpose((1, 2, 0))

    im = (cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0) > 0) * 1.0
    im = cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0)

    return im
    
def transformation_from_points(points1, points2):
    """
    Return an affine transformation [s * R | T] such that:
        sum ||s*R*p1,i + T - p2,i||^2
    is minimized.
    """
    # Solve the procrustes problem by subtracting centroids, scaling by the
    # standard deviation, and then using the SVD to calculate the rotation. See
    # the following for more details:
    #   https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem

    points1 = points1.astype(numpy.float64)
    points2 = points2.astype(numpy.float64)

    c1 = numpy.mean(points1, axis=0)
    c2 = numpy.mean(points2, axis=0)
    points1 -= c1
    points2 -= c2

    s1 = numpy.std(points1)
    s2 = numpy.std(points2)
    points1 /= s1
    points2 /= s2

    U, S, Vt = numpy.linalg.svd(points1.T * points2)

    # The R we seek is in fact the transpose of the one given by U * Vt. This
    # is because the above formulation assumes the matrix goes on the right
    # (with row vectors) where as our solution requires the matrix to be on the
    # left (with column vectors).
    R = (U * Vt).T

    return numpy.vstack([numpy.hstack(((s2 / s1) * R,
                                       c2.T - (s2 / s1) * R * c1.T)),
                         numpy.matrix([0., 0., 1.])])

def read_im_and_landmarks(image):
    im = image
    im = cv2.resize(im,None,fx=1, fy=1, interpolation = cv2.INTER_LINEAR)
    im = cv2.resize(im, (im.shape[1] * SCALE_FACTOR,
                         im.shape[0] * SCALE_FACTOR))
    s = get_landmarks(im)

    return im, s

def warp_im(im, M, dshape):
    output_im = numpy.zeros(dshape, dtype=im.dtype)
    cv2.warpAffine(im,
                   M[:2],
                   (dshape[1], dshape[0]),
                   dst=output_im,
                   borderMode=cv2.BORDER_TRANSPARENT,
                   flags=cv2.WARP_INVERSE_MAP)
    return output_im

def correct_colours(im1, im2, landmarks1):
    blur_amount = COLOUR_CORRECT_BLUR_FRAC * numpy.linalg.norm(
                              numpy.mean(landmarks1[LEFT_EYE_POINTS], axis=0) -
                              numpy.mean(landmarks1[RIGHT_EYE_POINTS], axis=0))
    blur_amount = int(blur_amount)
    if blur_amount % 2 == 0:
        blur_amount += 1
    im1_blur = cv2.GaussianBlur(im1, (blur_amount, blur_amount), 0)
    im2_blur = cv2.GaussianBlur(im2, (blur_amount, blur_amount), 0)

    # Avoid divide-by-zero errors.
    im2_blur += (128 * (im2_blur <= 1.0)).astype(im2_blur.dtype)

    return (im2.astype(numpy.float64) * im1_blur.astype(numpy.float64) /
                                                im2_blur.astype(numpy.float64))

## Extract landmarks from the face images

In [ ]:
# set output folder
outpath_folder = os.path.join(outpath,"landmarks")
# create dir
if not os.path.exists(outpath_folder):
    os.makedirs(outpath_folder)

# extract landmarks
i = 0
undet_faces = []
landmks = []
for f in img_faces:
    # landmarks = get_landmarks(f)
    rects = detector(f, 1)
    if len(rects) == 1: 
        landmarks = numpy.matrix([[p.x, p.y] for p in predictor(f, rects[0]).parts()])
        landmks.append(landmarks)
        image_with_landmarks = annotate_landmarks(f, landmarks)
        cv2.imwrite((outpath_folder+'/face_%02d_landmarks.png' % i), image_with_landmarks)
    else:
        print('Extraction of landmarks failed on image n. %d' % i)
        show_img(f, 'image %d' % i)
        undet_faces.append(i)
    i = i+1

### Question
Landmarks extraction can fail on some images, due for example to noise. 
Define a pre-processing method to apply to those images in order to be able to extract the landmarks.

# Part 3 - Align images
This part is devoted to align all face images extracted. Landmarks will be used for estimating a geometric transformation for warping an image to another.

Below an example of three aligned faces. The first was used as reference and the other two were aligned on the first. After alignment, all images have the same size.

<table><tr><td><img src="examples/face_00.png" style="width: 200px;"/></td><td><img src="examples/face_01_warped.png" style="width: 200px;"/></td><td><img src="examples/face_02_warped.png" style="width: 200px;"/></td></tr></table>

### Question
The code below performs the alignment taking an image as master and aligning all the others to this.
Have a look to the code and try to understand what it does.

You can now experiment for example by 
- modifying the master image
- changing the set of facial landmarks used for the alignment
- ...


In [ ]:
# face alignment
# set output folder
outpath_folder = os.path.join(outpath,"align")
# create dir
if not os.path.exists(outpath_folder):
    os.makedirs(outpath_folder)

fmaster = img_faces[0]
lmaster = landmks[0]

i = 1
for f, l in zip(img_faces[1:],landmks[1:]):
    M = transformation_from_points(lmaster[ALIGN_POINTS],
                                   l[ALIGN_POINTS])
    mask = get_face_mask(f, l)
    warped_mask = warp_im(mask, M, fmaster.shape)
    combined_mask = numpy.max([get_face_mask(fmaster, lmaster), warped_mask],
                          axis=0)
    warped_im2 = warp_im(f, M, fmaster.shape)
    cv2.imwrite((outpath_folder+'/face_%02d_align.png' % i), warped_im2)
    i = i+1
    

## Optional: Face swapping

Being able to establish correspondences among face images, allows one to do operations such as face swapping.
See for example: https://matthewearl.github.io/2015/07/28/switching-eds-with-python/

Implement a face swapping operator and apply it to two images.

# Part 4 - Face segmentation
The face images that were extracted have different backgrounds. One could be interested in performing a segmentation of the image in order to extract the face as a region separated from the background. After segmentation, the background can be processed separately from the face region.

### Question
Tune the segmentation algorithm. For example you can change:
- initial shape of the snake 
- blur applied to the image (`gaussian(img, ...)`)
- parameters defining the internal energy of the snake 

### Optional
Perform the face segmentation using another approach. You can for example exploit the position of the facial landmarks previously extracted. Run the algorithm on all images.

In [ ]:
import math
import matplotlib.path
import numpy as np

def pixels_inside_contour(polygon, size):
    
    i = np.arange(0, size[0])
    j = np.arange(0, size[1])
    xv, yv = np.meshgrid(i, j, indexing='ij')
    points = np.hstack((xv.reshape((-1,1)), yv.reshape((-1,1))))

    path = matplotlib.path.Path(polygon)
    mask = path.contains_points(points)
    mask.shape = xv.shape
    
    return mask

In [ ]:
# test face segmentation
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path
from skimage.color import rgb2gray
from skimage import data
from skimage.filters import gaussian
from skimage.segmentation import active_contour

# set output folder
outpath_folder = os.path.join(outpath,"segm")
# create dir
if not os.path.exists(outpath_folder):
    os.makedirs(outpath_folder)
    

# convert to grayscale
img = cv2.cvtColor(img_faces[0], cv2.COLOR_BGR2GRAY)

# define the initial contour
s = np.linspace(0, 2*np.pi, 400)
r = 60 + 100*np.sin(s)
c = 60 + 50*np.cos(s)
init = np.array([r, c]).T

# compute an active contour segmentation 
snake = active_contour(gaussian(img, 2.0),
                       init, alpha=0.015, beta=75, gamma=0.001,
                       coordinates='rc')

# plot the init and final contour
fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(img, cmap=plt.cm.gray)
ax.plot(init[:, 1], init[:, 0], '--r', lw=3)
ax.plot(snake[:, 1], snake[:, 0], '-b', lw=3)
ax.set_xticks([]), ax.set_yticks([])
ax.axis([0, img.shape[1], img.shape[0], 0])
plt.savefig((outpath_folder+'/face_%02d_segm.png' % 0))
plt.show()


# get the pixels inside the contour and generate a binary mask
mask = pixels_inside_contour(snake, img.shape)
show_img(mask, "mask")

### Question
In this part you will process the image background.
- Set the background pixel to an arbitrary value
- Blur the background (e.g., by simulating a Bokeh effect https://en.wikipedia.org/wiki/Bokeh)
- Blur the background with an increasing value of blur with respect to the distance from the face (you may use the `spipy` function `distance_transform_edt`)
- Substitute the background with another one. Use a custom image as background. You may want to smooth the transition between foreground and background in order to reduce the effect of imprecisions in segmentation
- ...

Process all images.

# Optional
Other possible developments:
- Compute eigenfaces (https://en.wikipedia.org/wiki/Eigenface). These are features obtained by a eigendecomposition of a set of aligned faces. These features can be used for performing face recognition.
In this case it is better to use images with blurred background or crop the images leaving only the face.
Example: https://scipy-lectures.org/packages/scikit-learn/auto_examples/plot_eigenfaces.html
- Knowing the position of facial landmarks (eyes, nose, mouth...) allows to perform some post-processing on the image. For example you can implement some "make-up" filters in order to perform some operations on the face. You can also add glasses, moustaches etc.
- Cartoonize the images. You can run some segmentation algorithms on the image and perform vector quantization (giving the average color of a region to all its pixels)
- ...